In [39]:
import ssl
import pandas as pd
import wandb

ssl._create_default_https_context = ssl._create_unverified_context
import os
from torch.utils.data import DataLoader
from torch import nn
from torchvision.io import read_image
from torchvision.transforms import v2
import torch
import torchvision.transforms as T
import json
import urllib
import requests
from PIL import Image
from io import BytesIO
import numpy as np
from sklearn.model_selection import train_test_split

In [40]:
dataframe = pd.DataFrame()
dataframe['image1'] = [f'image 1 row {i}' for i in range(20)]
dataframe['image2'] = [f'image 2 row {i}' for i in range(20)]
dataframe['image3'] = [f'image 3 row {i}' for i in range(20)]
dataframe['image4'] = [f'image 4 row {i}' for i in range(20)]
dataframe['image5'] = [f'image 5 row {i}' for i in range(20)]
dataframe['target'] = [f'target 5 row {i}' for i in range(20)]
dataframe

,image1,image2,image3,image4,image5,target
0,image 1 row 0,image 2 row 0,image 3 row 0,image 4 row 0,image 5 row 0,target 5 row 0
1,image 1 row 1,image 2 row 1,image 3 row 1,image 4 row 1,image 5 row 1,target 5 row 1
2,image 1 row 2,image 2 row 2,image 3 row 2,image 4 row 2,image 5 row 2,target 5 row 2
3,image 1 row 3,image 2 row 3,image 3 row 3,image 4 row 3,image 5 row 3,target 5 row 3
4,image 1 row 4,image 2 row 4,image 3 row 4,image 4 row 4,image 5 row 4,target 5 row 4
5,image 1 row 5,image 2 row 5,image 3 row 5,image 4 row 5,image 5 row 5,target 5 row 5
6,image 1 row 6,image 2 row 6,image 3 row 6,image 4 row 6,image 5 row 6,target 5 row 6
7,image 1 row 7,image 2 row 7,image 3 row 7,image 4 row 7,image 5 row 7,target 5 row 7
8,image 1 row 8,image 2 row 8,image 3 row 8,image 4 row 8,image 5 row 8,target 5 row 8
9,image 1 row 9,image 2 row 9,image 3 row 9,image 4 row 9,image 5 row 9,target 5 row 9


In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [42]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data_frame):
        self.df = data_frame

    def __getitem__(self, index):
        outfit = self.df.iloc[index]
        # print(f'accessoire shape: {img_accessoire.shape}; path={outfit["Accessoire_imagePath"]}')
        # print(f'innerwear shape: {img_innerwear.shape}; path={outfit["Innerwear_imagePath"]}')
        return (outfit['image1'],
                outfit['image2'],
                outfit['image3'],
                outfit['image4'],
                outfit['image5']), outfit['target']

    def __len__(self):
        return self.df.shape[0]

In [71]:
def proccess(X, y):
    #number_of_rows = X[0].shape[0]
    number_of_rows = len(X[0])
    print(f'number of rows: {number_of_rows}')
    for index in range(number_of_rows):
        print(
            f'input into model: {X[0][index]}; {X[1][index]}; {X[2][index]}; {X[3][index]}; {X[4][index]}, should be predicted to: {y[index]}')

        # extracted_features = torch.cat(
        #     (
        #         create_linear_input(self._feature_extraction_model.forward(X[0][index]),
        #                             model_config['use_n_blocks'],
        #                             False),
        #         create_linear_input(self._feature_extraction_model.forward(X[1][index]),
        #                             model_config['use_n_blocks'],
        #                             False),
        #         create_linear_input(self._feature_extraction_model.forward(X[2][index]),
        #                             model_config['use_n_blocks'],
        #                             False),
        #         create_linear_input(self._feature_extraction_model.forward(X[3][index]),
        #                             model_config['use_n_blocks'],
        #                             False),
        #         create_linear_input(self._feature_extraction_model.forward(X[4][index]),
        #                             model_config['use_n_blocks'], False)
        # ),
        # dim=1
        # )


dataloader = torch.utils.data.DataLoader(CustomDataset(dataframe), batch_size=7,
                                         shuffle=True, num_workers=0)
for batch, (X, y) in enumerate(dataloader):
    print(y)
    proccess(X, y)
    break


('target 5 row 18', 'target 5 row 6', 'target 5 row 9', 'target 5 row 12', 'target 5 row 1', 'target 5 row 11', 'target 5 row 3')
number of rows: 7
input into model: image 1 row 18; image 2 row 18; image 3 row 18; image 4 row 18; image 5 row 18, should be predicted to: target 5 row 18
input into model: image 1 row 6; image 2 row 6; image 3 row 6; image 4 row 6; image 5 row 6, should be predicted to: target 5 row 6
input into model: image 1 row 9; image 2 row 9; image 3 row 9; image 4 row 9; image 5 row 9, should be predicted to: target 5 row 9
input into model: image 1 row 12; image 2 row 12; image 3 row 12; image 4 row 12; image 5 row 12, should be predicted to: target 5 row 12
input into model: image 1 row 1; image 2 row 1; image 3 row 1; image 4 row 1; image 5 row 1, should be predicted to: target 5 row 1
input into model: image 1 row 11; image 2 row 11; image 3 row 11; image 4 row 11; image 5 row 11, should be predicted to: target 5 row 11
input into model: image 1 row 3; image 2 r

In [70]:
torch.squeeze(torch.tensor([
    [
        [
            [
                [
                    1
                ]
            ]
        ]
    ],
    [
        [
            [
                [
                    2
                ]
            ]
        ]
    ]
]))

tensor([1, 2])